In [2]:
import numpy as np
import pandas as pd
from sympy import *
import math
import os

In [3]:
#Input Pressure and Temperature values
P = float(input("Enter pressure in psia:")) # psia
T = float(input("Enter Temperature in Farenheit: ")) # Farenheit

In [4]:
#Import Components Properties
component_properties = pd.read_excel("components.xlsx")
component_properties

,component,composition,Pci,Tci,wi,Mwi,Vci
0,N2,0.0200,493.1,227.49,0.040,28.013,0.0510
1,C1,0.8866,666.4,343.33,0.008,16.043,0.0988
2,C2,0.0492,706.5,549.92,0.098,30.070,0.0783
3,C3,0.0246,616.0,666.06,0.152,44.097,0.0727
4,nC4,0.0098,550.6,765.62,0.193,58.123,0.0703
5,nC5,0.0098,488.6,845.80,0.251,72.151,0.0675


In [5]:
#Import binary coefficients
d= pd.read_excel("binary.xlsx").values
d

array([[0.    , 0.018 , 0.039 , 0.046 , 0.047 , 0.048 ],
       [0.018 , 0.    , 0.005 , 0.01  , 0.0145, 0.0182],
       [0.039 , 0.005 , 0.    , 0.0017, 0.0032, 0.0048],
       [0.046 , 0.01  , 0.0017, 0.    , 0.0012, 0.0024],
       [0.047 , 0.0145, 0.0032, 0.0012, 0.    , 0.0008],
       [0.048 , 0.0182, 0.0048, 0.0024, 0.0008, 0.    ]])

In [6]:
Pci = component_properties['Pci'] 
Tci = component_properties['Tci']

Pri= P/Pci
Tri= (T+459.67 )/Tci
print('Reduced Pressure,  Pri:\n', Pri)
print('Reduced Temperature, Tri:\n',Tri)

Reduced Pressure,  Pri:
 0    1.520990
1    1.125450
2    1.061571
3    1.217532
4    1.362150
5    1.534998
Name: Pci, dtype: float64
Reduced Temperature, Tri:
 0    1.712910
1    1.134972
2    0.708594
3    0.585037
4    0.508960
5    0.460712
Name: Tci, dtype: float64


In [7]:
wi = component_properties['wi']
w_a = 0.42748020
w_b = 0.08664035
mi = 0.48+1.574*wi-0.176*wi**2
        
Ai =(w_a*(1+mi*(1-Tri**0.5))**2*(Pri/Tri**2)).replace(np.nan,0).to_numpy()
Bi = ((w_b*Pri)/Tri).to_numpy()
print('Ai:',Ai)
print('Bi:',Bi)

Ai: [0.15355681 0.3498256  1.09375913 2.07505954 3.36080423 5.04568727]
Bi: [0.07693285 0.08591347 0.12979915 0.18030889 0.23187907 0.28866804]


In [8]:
#Calculating A and B
c = component_properties['composition'].to_numpy()
n= len(c)

B =  np.sum(c * Bi)

A=0
for i in range (n):
    for j in range (n):
        A += (1-d[i,j])*(Ai[i]*Ai[j])**0.5*c[i]*c[j] 
        
print('A :', A)
print('B :', B)

A : 0.4328099111773391
B : 0.0936326168492695


In [9]:
#Calculating Z 

m1 = 0
m2 = 1
p=1
q=(m1+m2-1)*B-1
r=(A+m1*m2*B**2-(m1+m2)*B*(B+1))
s=-A*B-m1*m2*B**2*(B+1)

Z = np.roots([p, q, r, s])

print('For Pressure:',P , 'and', 'Temperature:',T )
print('                          A:', round(A,4))
print('                          B:', round(B,4))
print('                          Z:', Z)
for i in range(len(Z)):
    if np.isreal(Z[i]):
        print('Without imaginery values, Z:', np.real(Z[i]))
        
print('                      Z_max:', max(Z))
print('                      Z_min:', min(Z))

For Pressure: 750.0 and Temperature: -70.0
                          A: 0.4328
                          B: 0.0936
                          Z: [0.50388514+0.j        0.24805743+0.1374512j 0.24805743-0.1374512j]
Without imaginery values, Z: 0.5038851411388346
                      Z_max: (0.5038851411388346+0j)
                      Z_min: (0.24805742943058284-0.13745120376332645j)


In [10]:

Ki = (1/Pri)*((5.37*(1+wi)*(1-(1/Tri))).apply(math.exp))

Ki 

0    6.719563
1    1.691359
2    0.083362
3    0.010207
4    0.001519
5    0.000250
dtype: float64

In [11]:
g_V = lambda v:sum(c*(Ki - 1)/(v*(Ki - 1)+1))
dg_V = lambda v:-sum(c*((Ki-1)**2)/((v*(Ki-1) + 1)**2))

A = sum(c*(Ki-1))
B = sum(c*(Ki-1)/Ki)

v = A/(A - B) 

while abs(g_V(v))>0.0001:
    v = v - g_V(v)/dg_V(v)

l = 1 - v

print(f"Composition in vapor phase is: {v}")
print(f"Composition in liquid phase is: {l}")

Composition in vapor phase is: 0.8162445401238981
Composition in liquid phase is: 0.18375545987610187


In [12]:
Xi = c/(1 + v*(Ki-1))
Yi = Xi*Ki

def Aa(c):
    return [sum(c[i]*c*(Ai[i]*Ai)**0.5) for i in range(len(c))]

Al = sum(Aa(Xi))
Av = sum(Aa(Yi))
Bl = sum(Xi*Bi)
Bv = sum(Yi*Bi)

print(f"A in Vapour phase {Av} and liquid phase {Al}")
print(f"B in Vapour phase {Bv} and liquid phase {Bl}")

A in Vapour phase 0.35445500554452064 and liquid phase 0.8877605590674149
B in Vapour phase 0.0865532181231482 and liquid phase 0.12507941337685463


In [13]:
m1 = 1+2**0.5
m2 = 1-2**0.5

#Vapour Phase 
p=1
q=(m1+m2-1)*Bv-1
r=(Av+m1*m2*Bv**2-(m1+m2)*Bv*(Bv+1))
s=-Av*Bv-m1*m2*Bv**2*(Bv+1)

Zv = np.roots([p, q, r, s])

print('Vapour Phase compressibility factor, Zv:', Zv)


print('Vapour Phase :', np.real(Zv))

#Liquid phase 
t=1
u=(m1+m2-1)*Bl-1
w=(Al+m1*m2*Bl**2-(m1+m2)*Bl*(Bl+1))
x=-Al*Bl-m1*m2*Bl**2*(Bl+1)

Zl = np.roots([t, u, w, x])
print('Liquid Phase compressibility factor, Zl:', Zl)

# for i in range(len(Zl)):
#     if np.isreal(Zl[i]):
#         print('Liquid Phase compressibility factor, Zl:', np.real(Zl[i]))


Vapour Phase compressibility factor, Zv: [0.73989304+0.j         0.08677687+0.15143567j 0.08677687-0.15143567j]
Vapour Phase : [0.73989304 0.08677687 0.08677687]
Liquid Phase compressibility factor, Zl: [0.33425361+0.58391168j 0.33425361-0.58391168j 0.20641337+0.j        ]


In [14]:
print(f'For Pressure: {P} psia  and Temperature: {T} Farenheit' )

print(f"Composition of Vapor phase : {v} and Liquid Phase: {l}")
print(f"A in Vapour phase {Av} and Liquid phase {Al}")
print(f"B in Vapour phase {Bv} and Liquid phase {Bl}")
print(f"Compressibility factor,  Vapour phase , Zv: {Zv} and Liquid phase, Zl: {Zl} ")

For Pressure: 750.0 psia  and Temperature: -70.0 Farenheit
Composition of Vapor phase : 0.8162445401238981 and Liquid Phase: 0.18375545987610187
A in Vapour phase 0.35445500554452064 and Liquid phase 0.8877605590674149
B in Vapour phase 0.0865532181231482 and Liquid phase 0.12507941337685463
Compressibility factor,  Vapour phase , Zv: [0.73989304+0.j         0.08677687+0.15143567j 0.08677687-0.15143567j] and Liquid phase, Zl: [0.33425361+0.58391168j 0.33425361-0.58391168j 0.20641337+0.j        ] 
